In [2]:
import re
from pathlib import Path
import pandas

In [4]:
data = Path("../data/lda_test_100_cll_progress.csv")

df = pandas.read_csv(data, index_col=0)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.decomposition import LatentDirichletAllocation

In [68]:
# unit_stop_words = text.ENGLISH_STOP_WORDS.union(['mg', 'ml', 'ul', 'dl', 'hrs', 'NUMmg', 'NUMml', 'NUMul', 'NUMhrs'])
unit_stop_words = ['mg', 'ml', 'ul', 'dl', 'hrs', 'and', 'with', 'no', 'to', 'is', 'of', 'has', 'on', 'for', 'the', 'he', 'she', 'in', 'this', 'was', 'or', 'by', 'not', 'as', 'at', 'his', 'her', 'does', 'yes', 'that', 'we', 'will', 'are', 'who', 'be', 'dr', 'if', 'could', 'patient', 'pt', 'take', 'from', 'there', 'kg', 'daily']

In [69]:
# tf_vectorizer = TfidfVectorizer(preprocessor=lambda x: re.sub(r'(\d)+', 'NUM', x.lower()), max_df=0.9, stop_words=unit_stop_words)
tf_vectorizer = CountVectorizer(preprocessor=lambda x: ''.join(c for c in x if not c.isdigit()).lower(), max_df=0.95, stop_words=unit_stop_words)
# tf_vectorizer = TfidfVectorizer(stop_words=unit_stop_words)
# tf_vectorizer = CountVectorizer(max_df=0.95, stop_words=unit_stop_words)
tf = tf_vectorizer.fit_transform(df.full_note_norm)
tf_feature_names = tf_vectorizer.get_feature_names()
lda = LatentDirichletAllocation(n_components=5, max_iter=10, learning_method='online', learning_offset=10.,random_state=0).fit(tf)

In [60]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [70]:
display_topics(lda, tf_feature_names, 10)

Topic 0:
transfer diff manual sigmoid medicine labslabs tablet see aemergency last
Topic 1:
sw home bed gait diet therapeutic minutes ambulation discharge therapy
Topic 2:
final history last tablet labs oral day see mouth continue
Topic 3:
tablet oral labs last history recent qh final mouth diff
Topic 4:
care sw requires history hha fim score assistance intern bronchitis


In [95]:
tkz = tf_vectorizer.build_tokenizer()
df2 = df.full_note_norm.apply(tkz)

In [213]:
import numpy as np
vocab = np.array([x for x in tf_vectorizer.vocabulary_.keys()])

In [223]:
type(tf)

scipy.sparse.csr.csr_matrix

In [117]:
tf.shape

(100, 6196)

In [112]:
import tmtoolkit

In [220]:
# tmtoolkit.topicmod.evaluate.metric_coherence_gensim(measure='u_mass', topic_word_distrib=lda.components_, dtm=tf, vocab=vocab, texts=df.full_note_norm.values, return_mean=True)
tmtoolkit.topicmod.evaluate.metric_coherence_mimno_2011(topic_word_distrib=lda.components_, dtm=tf, normalize=True, return_mean=True)

-2.800670926968757